In [22]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
import itertools as it
import missing_bayesian_mixture as mbm
import os
import denoising_ae
import category_dae
import tensorflow as tf
import missForest
import missForest_GP

from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(mbm)
imp.reload(category_dae)
imp.reload(missForest)
imp.reload(missForest_GP)
imp.reload(denoising_ae)
imp.reload(gmm_impute)

<module 'gmm_impute' from '/mnt/disk1/MIMIC/ts-baselines/timeless-imputation/gmm_impute.py'>

In [11]:
from IPython.display import HTML
def df_view(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))

In [17]:
dsets = datasets.datasets()

+++ Importing Soybean
+++ Importing BreastCancer
+++ Importing LetterRecognition
+++ Importing Shuttle
+++ Importing Servo
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing BostonHousing


In [18]:
tests_to_perform = []
for b in dsets.items():
    def do_mcar_rows(dataset_, proportion):
        return utils.mcar_rows(dataset_, proportion**.5, proportion**.5)
    def do_mar_rows(dataset_, proportion):
        return utils.mar_rows(dataset_, proportion**.5, proportion**.5,
                              deciding_missing_proportion=0.2)
    def do_mnar_rows(dataset_, proportion):
        return utils.mnar_rows(dataset_, proportion**.5, proportion**.5,
                            missing_proportion_nonrandom=0.0)

    for c in [(datasets.memoize(utils.mcar_total), 'MCAR_total'),
              (datasets.memoize(do_mcar_rows), 'MCAR_rows'),
              (datasets.memoize(do_mnar_rows), 'MNAR_rows'),
              (datasets.memoize(do_mar_rows), 'MAR_rows')]:
        for d in [.1, .3, .5, .7, .9]:
            for e in ['mean_std']:
                tests_to_perform.append((b, c, d, e))
del b, c, d, e

In [ ]:
_ds = dict((x, dsets[x]) for x in ["BostonHousing", "Servo", "Ionosphere"
    # "Soybean", "BreastCancer"
])
baseline = datasets.benchmark([#'MICE': datasets.memoize(utils.impute_mice),
        ('mean', missForest.impute),
        ('GMM', lambda log, d, full_data: missForest.impute(
            log, d, full_data, max_iterations=0,
            initial_impute=mbm.mf_initial_impute)),
        ('Missforest_mult', datasets.memoize(utils.impute_missforest)),
        ('MICE', datasets.memoize(utils.impute_mice)),
        ('GP_mog_prior', lambda log_path, d, full_data: missForest.impute(
                log_path, d, full_data)),
        ('GP_KNN_prior', lambda log_path, d, full_data: missForest.impute(
                log_path, d, full_data)),
        ('GP_mog', lambda log_path, d, full_data: missForest.impute(
            log_path, d, full_data, sequential=False, print_progress=True,
            predictors=(missForest_GP.UncertainGPClassification,
                        missForest_GP.UncertainGPRegression),
            optimize_gp=False, use_previous_prediction=False,
            ARD=False, impute_name_replace=('GP', 'GMM'), max_iterations=1)),
        ('GP_KNN', lambda log_path, d, full_data: missForest.impute(
            log_path, d, full_data, sequential=False, print_progress=True,
            predictors=(missForest_GP.KNNGPClassification,
                        missForest_GP.KNNGPRegression),
            optimize_gp=True, use_previous_prediction=False,
            ARD=True, n_neighbours=5, knn_type='kernel_avg', max_iterations=1)),
        #('GP_KNN_noopt', lambda log_path, d, full_data: missForest.impute(
        #    log_path, d, full_data, sequential=False, print_progress=True,
        #    predictors=(missForest_GP.KNNGPClassification,
        #                missForest_GP.KNNGPRegression),
        #    optimize_gp=False, use_previous_prediction=False,
        #    ARD=True, n_neighbours=5, knn_type='kernel_avg', max_iterations=1)),

], _ds, tests_to_perform, do_not_compute=True)

In [ ]:
baseline.loc["mean_std", :].drop(["LetterRecognition", "Shuttle", "Soybean", "BreastCancer", "Servo"], axis=0)# .drop([("NRMSE", "MF_py"), ("PFC", "MF_py"), ("RMSE", "MF_py")], axis=1)  # 

In [170]:
imp.reload(mbm)
bayesgmm = datasets.benchmark({'BGMM_50': (lambda p, d, full_data: mbm.impute_bayes_gmm(
    p, d, full_data=full_data, number_imputations=100, n_components=50))
                              }, dsets, do_not_compute=True)

In [295]:
results = pd.concat([baseline, bayesgmm, nnresults], axis=1).loc['mean_std']
results = results.drop([('total_cats', 'MissForest'), ('total_cats', 'MLP_2x256_SGD')], axis=1)
results.sort_index(level=[0,1], inplace=True, axis=1)
results

In [319]:
# Export the results to LaTeX

s = (("""\\documentclass[12pt,a3paper,landscape,oneside]{article}
\\usepackage[utf8]{inputenc}
\\usepackage[english]{babel}
\\usepackage{setspace}
\\usepackage{parskip}
\\usepackage{color}
\\usepackage{amssymb,amsmath,amsfonts}
\\usepackage{graphicx}
\\usepackage{hyperref}
\\hypersetup{colorlinks,breaklinks,urlcolor=blue,linkcolor=blue} % Set link colors throughout the document
\\usepackage[left=2cm,right=2cm,top=2cm,bottom=2cm]{geometry}
\\usepackage{enumitem}   
\\usepackage{import}
\\usepackage{multicol}

\\usepackage{csquotes}
\\begin{document}\n""" + results.to_latex() + "\n\\end{document}")
     .replace("\\toprule", "\\hline")
     .replace("\\bottomrule", "\\hline")
     .replace("\\midrule", "\\hline"))

with open("measurements.tex", "w") as f:
    f.write(s)


In [ ]:
from rpy2.robjects.packages import importr
import rpy2.robjects
import rpy2.robjects.pandas2ri
import rpy2.robjects.numpy2ri
import rpy2.rinterface
import numpy as np
import pickle_utils as pu
import pandas as pd
import collections
import utils
import os
import collections
import unittest

rpy2.robjects.pandas2ri.activate()
rpy2.robjects.numpy2ri.activate()
py2ri = rpy2.robjects.pandas2ri.py2ri
ri2py = rpy2.robjects.pandas2ri.ri2py
R = rpy2.robjects.r

base = importr("base")
R_utils = importr("utils")
mlbench = importr("mlbench")
randomForest = importr("randomForest")

In [77]:
dsets_label = datasets.datasets(exclude_labels=False)
path = "impute_benchmark"
for algo_name in ["BayesGMM", "MICE", "MissForest"]:
    for data_name, (full_data, cat_keys) in dsets_label.items():
        if data_name == 'BostonHousing':
            continue

        for ampute_fun_name in ['MCAR_total', 'MCAR_rows']:
            for proportion in [.1, .3, .5, .7, .9]:
                for norm_type in ['mean_std']:  # 'min_max']:
                    amputed_name = '{:s}_{:s}_{:.1f}'.format(
                        data_name, ampute_fun_name, proportion) 
                    imputed_path = os.path.join(
                        path, 'imputed_{:s}_{:s}.pkl.gz'.format(
                            algo_name, amputed_name))
                    if not os.path.exists(imputed_path):
                        continue
                    amputed_data = pu.load(os.path.join(path, 'amputed_'+amputed_name+'.pkl.gz'))
                    imputed_data = pu.load(imputed_path)
                    (_ad, _id), moments = utils.normalise_dataframes(
                        amputed_data, imputed_data, method=norm_type)
                    
                    to_learn = pd.concat([_id, full_data[["Class"]]], axis=1)
                    test_mask = np.random.rand(len(to_learn)) < 0.2
                    training_set = to_learn[~test_mask]
                    test_set = to_learn[test_mask]
                    
                    


+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing BostonHousing


AssertionError: All dataframes must have the same columns